In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

spark = SparkSession.builder.appName('Teste').getOrCreate()

df_titles = spark.read.load(r'C:\Users\joth1\Documents\z_projetinho_pyspark\titles.csv',     header=True,  inferSchema=True,    sep=',',    format='csv')
df_titles.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

#### Podemos notar que dentro da tabela existem 10MMM de filmes, nesse trecho abaixo trago tambem as colunas tratadas

In [33]:
# ALTERE ESSA PARTE 
colunas_que_quero_renomear = ['tconst', 'titleType','isAdult', 'genres', 'runtimeMinutes', 'startYear']
novos_nomes_colunas = ['id_filme', 'tipo_filme', 'fl_filme_adulto', 'genero', 'tempo_filme', 'ano_lancamento']

# Cria um dictionary juntando as duas listas.
# Ou seja eu crio uma estrutura {nome_antiga_coluna : nome_nova_coluna}
# e reuso ela no for abaixo para substituir o nome das colunas.  NÃO ALTERE A PARTE ABAIXO
novas_colunas = {antiga_coluna : nova_coluna for antiga_coluna, nova_coluna in zip(colunas_que_quero_renomear, novos_nomes_colunas)}

for coluna_antiga, coluna_nova in novas_colunas.items():
    if coluna_antiga in df_titles.columns:
        df_titles = df_titles.withColumnRenamed(coluna_antiga, coluna_nova)

df_titles.show()
df_titles.createOrReplaceTempView('titulos')


contador_titles = spark.sql("""SELECT COUNT(*) AS QTD FROM titulos""")
contador_titles.show()

+---------+----------+--------------------+--------------------+---------------+--------------+-------+-----------+--------------------+
| id_filme|tipo_filme|        primaryTitle|       originalTitle|fl_filme_adulto|ano_lancamento|endYear|tempo_filme|              genero|
+---------+----------+--------------------+--------------------+---------------+--------------+-------+-----------+--------------------+
|tt0000001|     short|          Carmencita|          Carmencita|              0|          1894|     \N|          1|   Documentary,Short|
|tt0000002|     short|Le clown et ses c...|Le clown et ses c...|              0|          1892|     \N|          5|     Animation,Short|
|tt0000003|     short|      Pauvre Pierrot|      Pauvre Pierrot|              0|          1892|     \N|          4|Animation,Comedy,...|
|tt0000004|     short|         Un bon bock|         Un bon bock|              0|          1892|     \N|         12|     Animation,Short|
|tt0000005|     short|    Blacksmith Scen

#### Nesse trecho abaixo trago os filmes presentes na netflix com seu titulo, diretor e tipo de filme 

In [34]:
df_netflix = spark.read.load(r'C:\Users\joth1\Documents\z_projetinho_pyspark\netflix_titles.csv',     header=True,  inferSchema=True,    sep=',',    format='csv')
df_netflix.createOrReplaceTempView('netflix')

df_netflix.show()
numero = df_netflix.count()
display(numero)

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|     Kirsten Johnson|                NULL|       United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|                NULL|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglan

8809

#### Aqui a ideia era verificar quantos % de filmes a netflix possui

In [40]:
df = spark.sql("""
SELECT 
          A.*
          ,B.*
FROM NETFLIX AS A 
INNER JOIN titulos AS B ON B.PRIMARYTITLE = A.TITLE
""")

df.show()

df = df.dropDuplicates(['title'])

df.createOrReplaceTempView('tabela_final')
contador = spark.sql("""SELECT COUNT(*) AS QTD FROM tabela_final""")
contador.show()

+-------+-------+--------------------+-------------------+--------------------+--------------------+------------------+------------+------+----------+--------------------+--------------------+---------+----------+--------------------+--------------------+---------------+--------------+-------+-----------+--------------------+
|show_id|   type|               title|           director|                cast|             country|        date_added|release_year|rating|  duration|           listed_in|         description| id_filme|tipo_filme|        primaryTitle|       originalTitle|fl_filme_adulto|ano_lancamento|endYear|tempo_filme|              genero|
+-------+-------+--------------------+-------------------+--------------------+--------------------+------------------+------------+------+----------+--------------------+--------------------+---------+----------+--------------------+--------------------+---------------+--------------+-------+-----------+--------------------+
|  s8335|TV Show

#### Nesse trecho, busquei identificar os principais filmes da Netflix com as melhores notas segundo as avaliações do IMDB, para verificar se a plataforma possui filmes de grande calibre. Comparei a bilheteria total dos filmes avaliados pelo IMDB e observei que a Netflix possui aproximadamente 1/5 do valor total das bilheterias dos filmes avaliados pelo IMDB.

In [73]:
df_imdb = spark.read.load(r'C:\Users\joth1\Documents\z_projetinho_pyspark\imdb_top_1000.csv',     header=True,  inferSchema=True,    sep=',',    format='csv')
# df_imdb.show()

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col, when, sum
from pyspark.sql.types import DoubleType


df_imdb = df_imdb.withColumn('gross', regexp_replace('gross', ',', ''))
df_imdb = df_imdb.withColumn('gross', col('gross').cast(DoubleType()))


df_imdb.createOrReplaceTempView('dados_imdb')


df_final = spark.sql(
"""
SELECT
    TRIM(A.TITLE) AS TITULO
    ,TRIM(A.DIRECTOR) AS DIRETOR
    ,TRIM(A.COUNTRY) AS PAIS
    ,A.DATE_ADDED AS DATA_QUE_FOI_ADICIONADO
    ,A.RELEASE_YEAR AS DATA_LANCAMENTO
    ,A.DURATION
    ,A.TIPO_FILME
    ,TRIM(A.GENERO) AS GENERO
    ,B.IMDB_RATING
    ,B.META_SCORE
    ,B.NO_OF_VOTES AS QTD_AVALIACOES
    ,CAST(B.GROSS AS DECIMAL(18, 2)) AS RECEITA_FILME
    ,CAST(B.GROSS AS BIGINT)/1000000 AS RECEITA_EM_MM

FROM tabela_final AS A 
INNER JOIN DADOS_IMDB AS B ON B.SERIES_TITLE = A.TITLE
ORDER BY IMDB_RATING DESC, NO_OF_VOTES DESC
""")
df_final.show()
numero = df_final.count()
display(numero)



soma_valor_netflix = df_final.select(sum("RECEITA_FILME")).collect()[0][0]
soma_valor = df_imdb.select(sum("gross")).collect()[0][0]


import locale
# Define a localidade para o Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
# Formata o valor como moeda brasileira
soma_valor = locale.currency(soma_valor, grouping=True)
soma_valor_netflix = locale.currency(soma_valor_netflix, grouping=True)

display('valor total de bilheteria de todos os filmes avaliados pelo IMDB: ' + soma_valor)
display('Valor total de bilheteria dos filmes netflix: ' + soma_valor_netflix)

+--------------------+--------------------+--------------------+-----------------------+---------------+--------+----------+--------------------+-----------+----------+--------------+-------------+-------------+
|              TITULO|             DIRETOR|                PAIS|DATA_QUE_FOI_ADICIONADO|DATA_LANCAMENTO|DURATION|TIPO_FILME|              GENERO|IMDB_RATING|META_SCORE|QTD_AVALIACOES|RECEITA_FILME|RECEITA_EM_MM|
+--------------------+--------------------+--------------------+-----------------------+---------------+--------+----------+--------------------+-----------+----------+--------------+-------------+-------------+
|        Pulp Fiction|   Quentin Tarantino|       United States|        January 1, 2019|           1994| 154 min|     movie|         Crime,Drama|        8.9|        94|       1826188| 107928762.00|   107.928762|
|The Lord of the R...|       Peter Jackson|New Zealand, Unit...|        January 1, 2020|           2003| 201 min|     movie|Action,Adventure,...|       

172

'valor total de bilheteria de todos os filmes avaliados pelo IMDB: R$ 55.059.273.035,00'

'Valor total de bilheteria dos filmes netflix: R$ 10.561.658.884,00'

#### Aqui eu peguei todos os filmes que a netflix tem que não foram registrados na tabela de filmes mundiais, caso alguém queira ver

In [76]:
df_somente_netflix = spark.sql("""
SELECT 
          A.*
          ,B.*
FROM NETFLIX AS A 
LEFT JOIN titulos AS B ON B.PRIMARYTITLE = A.TITLE
WHERE B.PRIMARYTITLE IS NULL
""")

df_somente_netflix.show()

numero = df_somente_netflix.count()
display('numero de filmes que a netflix possui que não foram registrados no catalogo de filmes/series existentes: ' + str(numero))

+-------+-------+--------------------+------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+--------+----------+------------+-------------+---------------+--------------+-------+-----------+------+
|show_id|   type|               title|          director|                cast|             country|        date_added|release_year|rating|duration|           listed_in|         description|id_filme|tipo_filme|primaryTitle|originalTitle|fl_filme_adulto|ano_lancamento|endYear|tempo_filme|genero|
+-------+-------+--------------------+------------------+--------------------+--------------------+------------------+------------+------+--------+--------------------+--------------------+--------+----------+------------+-------------+---------------+--------------+-------+-----------+------+
|  s2482|  Movie|   #FriendButMarried|     Rako Prijanto|Adipati Dolken, V...|           Indonesia|      May 21, 20